In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/DL_project/Song_emotion_compare/cat_proba_lyrics.csv')
df.mean()[:5]

<ipython-input-89-c583033134da>:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  list(df.mean()[:5])


[0.08725205660623461,
 0.15137762897081344,
 0.14268008641334992,
 0.2445885912322183,
 0.3741016367773835]

#코드 제작

In [ ]:
import torch
import pandas as pd
import numpy as np
import torch.nn.functional as F


def cos_recommend():
  # 우리가 만들어둔 노래 가사들을 감정분석 돌린 결과 csv파일을 읽어온다.
  df = pd.read_csv('/content/drive/MyDrive/DL_project/Song_emotion_compare/cat_proba_lyrics.csv')
  df = df.rename(columns={df.columns[5]:'song'})  

  # 입력 예시 (사용자와의 챗봇 대화를 통해서 상대방의 감정을 얻어낸 값을 user_emotion에에 넣으면 됨.)
  user_emotion = torch.tensor([0.1, 0.2, 0.2, 0.1, 0.4]) # 기분 예시
  
  # 우리의 노래 가사 분석 데이터를 tensor type으로 변환
  lyric_emotion = torch.from_numpy(df.values[:,:5].astype(float))
  
  #코사인 유사도 분석 함수를 적용
  df['similarity'] = pd.Series(F.cosine_similarity(user_emotion,lyric_emotion))
  
  # 기분전환 노래를 위한 감정 변환환 값
  mean_point = df.mean(numeric_only=True)[:5]
  user_emotion = user_emotion - 2*(user_emotion -mean_point)
  user_emotion = np.array([0 if i<0  else i for i in user_emotion])
  user_emotion = torch.tensor(user_emotion /sum(user_emotion))
  # 노래와 유저기분의 차이값 저장
  df['diff'] = pd.Series(F.cosine_similarity(user_emotion, lyric_emotion))

  #랜덤 적용
  random_add = np.random.rand(len(df['similarity']))/10
  df['similarity']  += random_add
  random_add = np.random.rand(len(df['similarity']))/10
  df['diff']  += random_add

  #유사도 높은 놈과 낮은 놈 출력
  print('유사도 높은 : ',df.iloc[np.argmax(df['similarity'])]['song'])
  print('반대 유사도 높은 : ',df.iloc[np.argmin(df['diff'])]['song'])

In [ ]:
cos_recommend()

tensor([0.0745, 0.1028, 0.0854, 0.3892, 0.3482], dtype=torch.float64)
유사도 높은 :  Off the menu - Sunshine
반대 유사도 높은 :  김진표 - 오롤롤롤로


#웹에 맞는 코드로 변환

In [ ]:
def cos_recommend(proba):
  # 우리가 만들어둔 노래 가사들을 감정분석 돌린 결과 csv파일을 읽어온다.
  df = pd.read_csv(os.path.join(filePath, 'data', 'cat_proba_lyrics.csv'))

  # 입력 예시 (사용자와의 챗봇 대화를 통해서 상대방의 감정을 얻어낸 값을 user_emotion에에 넣으면 됨.)
  user_emotion = torch.tensor(proba) # 기분 예시
  
  # 우리의 노래 가사 분석 데이터를 tensor type으로 변환
  lyric_emotion = torch.from_numpy(df.values[:,:5].astype(float))
  
  #코사인 유사도 분석 함수를 적용
  df['similarity'] = pd.Series(F.cosine_similarity(user_emotion,lyric_emotion))
  
  # 기분전환 노래를 위한 감정 변환환 값
  mean_point = df.mean(numeric_only=True)[:5]
  user_emotion = user_emotion - 2*(user_emotion -mean_point)
  user_emotion = np.array([0 if i<0  else i for i in user_emotion])
  user_emotion = torch.tensor(user_emotion /sum(user_emotion))
  # 노래와 유저기분의 차이값 저장
  df['diff'] = pd.Series(F.cosine_similarity(user_emotion, lyric_emotion))

  #랜덤 적용
  random_add = np.random.rand(len(df['similarity']))/10
  df['similarity']  += random_add
  random_add = np.random.rand(len(df['diff']))/10
  df['diff']  += random_add

  #유사도 높은 놈과 낮은 놈 출력
  return df.iloc[np.argmax(df['simillarity'])]['song'], df.iloc[np.argmin(df['diff'])]['song']